# 定数変更

In [1]:
#!sed --help

!sed -i s/"'gna_'  : 16"/"'gna_'  : 4"/ ./elecpy/cell/luorudy/const.py 

#!cat elecpy/cell/luorudy/const.py 
!cat elecpy/cell/luorudy/const.py  | grep gna_
#!cat elecpy/cell/luorudy/const.py  | grep gkr_
#!cat elecpy/cell/luorudy/const.py  | grep gks_

  'gna_'  : 4         , # Max. Conductance of the Na Channel (mS/uF)


# シミュレーション実行

In [2]:
import copy
import time
import os, sys
import shutil
import json
import numpy as np

import chainer
from chainer import cuda

from matplotlib import animation, rc
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import elecpy.elecpy as elp
from elecpy.util.cmap_bipolar import bipolar
from elecpy.elecpySession import ElecpySession

from IPython.display import HTML

from scipy.signal import argrelmax
from numba.decorators import autojit

%matplotlib inline


In [3]:
path_org = "/mnt/recordings/SimulationResult/20180806-3/"
path_dst = "/mnt/recordings/SimulationResult/20180816-1/"

In [4]:
# load sample parameter
with open (os.path.join(path_org, 'sim_params.json'),'r') as f:
    sim_params = json.load(f)

sim_params['restart'] = {}
sim_params['restart']['count'] = 2000
sim_params['restart']['source'] = path_org
    
# modify parameters
sim_params['time']['end'] = 200
sim_params["stimulation"] = {}
#sim_params["stimulation"]["extracellular"][0]["start"] = 0.
#sim_params["stimulation"]["extracellular"][0]["duration"] = 10.

sim_params['geometory']['height'] = 200
sim_params['geometory']['width'] = 200

sim_params['log']['path'] = path_dst
sim_params['log']['cnt'] = 1000


print json.dumps(sim_params, indent=4)


{
    "cell_type": "luorudy", 
    "log": {
        "path": "/mnt/recordings/SimulationResult/20180816-1/", 
        "cnt": 1000
    }, 
    "stimulation": {}, 
    "time": {
        "end": 200, 
        "udt": 0.001
    }, 
    "geometory": {
        "width": 200, 
        "ds": 0.015, 
        "height": 200
    }, 
    "restart": {
        "count": 2000, 
        "source": "/mnt/recordings/SimulationResult/20180806-3/"
    }
}


In [5]:
g = elp.sim_generator(sim_params)

In [6]:
os.mkdir(sim_params['log']['path'] )

In [7]:
with open(os.path.join(path_dst, 'sim_params.json'),'w') as f: json.dump(sim_params, f, indent=4)
out = []
while True:
    img = g.next()
    if img is False: break
    out.append(np.copy(img))

elecpy simulation start!


/root/.pyenv/versions/anaconda2-4.3.1/lib/python2.7/site-packages/chainer/cuda.py:84: UserWarning: cuDNN is not enabled.
Please reinstall CuPy after you install cudnn
(see https://docs-cupy.chainer.org/en/stable/install.html#install-cupy-with-cudnn-and-nccl).
  'cuDNN is not enabled.\n'


Stimulation settings ...done
Allocating data... ...done
Initializing data... ...done
Building PDE system ... ...done
Main loop start!
------------------200.0/200mselecpy done


In [16]:
import glob
import commands
def combine_restart( path_org, path_dst, restart=0, verbose=False, safe=False):
    keys = ['phie', 'vmem', 'cell']
    for key in keys:
        
        data_dst =sorted(glob.glob(path_dst+key+'_*'))
        for i, blob in enumerate(data_dst):
            cmd = 'mv {0} {1}'.format( blob, blob+'.org')
            #if key != 'cell': cmd+='.npy'
            if verbose:
                print cmd
            if not safe:
                commands.getoutput(cmd)

        for i, blob in enumerate(data_dst):
            cmd = 'cp -r {0} {1}'.format( blob+'.org', path_dst+key+'_{0:0>4}'.format(restart+i))
            if key != 'cell': cmd+='.npy'
            if verbose:
                print cmd
            if not safe:
                commands.getoutput(cmd)
                
        for i, blob in enumerate(data_dst):
            cmd = 'rm -r {0}'.format( blob+'.org')
            if verbose:
                print cmd
            if not safe:
                commands.getoutput(cmd)
        
        data_org = sorted(glob.glob(path_org+key+'_*'))
        for i, blob in enumerate(data_org):
            if i <= restart:
                cmd = 'cp -r {0} {1}'.format( blob, path_dst )
                if verbose:
                    print cmd
                if not safe:
                    commands.getoutput(cmd)
        


In [18]:
combine_restart(path_org, path_dst, restart=sim_params['restart']['count'])#, verbose=True, safe=True)